# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from scipy import stats
from scipy.stats import linregress

from citipy import citipy
from datetime import date
import scipy.stats as st


# Import API key
from api_keys import weather_api_key
from citipy import citipy



In [2]:
#create a list of lattitute and longitiutde and cities
lat_range = (-90,90)
lon_range = (-180,180)

cities = []
lat_lon = []
unique_cities = []

#import cities datafile 
cities_pd = pd.read_csv("../output_data/cities.csv")
cities_pd.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05



## Generate Cities List

In [3]:
#lat and long combination 
lat = np.random.uniform(lat_range[0], lat_range[1],size = 2000)
lon = np.random.uniform(lon_range[0], lon_range[1],size = 2000)

#map the lat and lon
lat_lon = zip(lat,lon)

#find the nearest city for a given coordinate

for coordinates in lat_lon:
    location_data = citipy.nearest_city(coordinates[0], coordinates[1])
    city = location_data.city_name
    country_name = location_data.country_code
    
    #if the location is unique add the new city into the cities list 
    if city not in unique_cities:
        unique_cities.append(city)
        cities.append(city)

print(len(cities))

751


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).



In [4]:
#URL
units = "metric"
url = "http://api.openweathermap.org/data/2.5/weather?"
API_url = f"{url}appid={weather_api_key}&units={units}&q="
query_url = url + "appid=" + weather_api_key + "&q="

In [5]:
#create lists to store the variable data 
lat = []
lng = []
temp = []
hum = []
cld = []
wind = []
dt =[]
city_list = []
country = []
no_data = []

In [6]:
cities

['umzimvubu',
 'jamestown',
 'mawlaik',
 'vaitupu',
 'ruston',
 'deming',
 'toyooka',
 'provideniya',
 'bengkulu',
 'punta arenas',
 'kaduna',
 'ushuaia',
 'kavaratti',
 'sibu',
 'arraial do cabo',
 'zhigansk',
 'cockburn town',
 'nikolskoye',
 'kazerun',
 'port elizabeth',
 'kysyl-syr',
 'otoka',
 'rikitea',
 'tessalit',
 'mataura',
 'pisco',
 'huescar',
 'hermanus',
 'tuktoyaktuk',
 'yellowknife',
 'almenara',
 'carnarvon',
 'parana',
 'mahebourg',
 'ruwi',
 'hobart',
 'touros',
 'fatikchhari',
 'altdorf',
 'diamantino',
 'tumannyy',
 'port alfred',
 'honningsvag',
 'avarua',
 'anadyr',
 'phan thiet',
 'norman wells',
 'husavik',
 'belawan',
 'albany',
 'zyryanka',
 'kapaa',
 'busselton',
 'bredasdorp',
 'cape town',
 'mys shmidta',
 'clonakilty',
 'ahipara',
 'egvekinot',
 'ormara',
 'abu dhabi',
 'umea',
 'hilo',
 'qaanaaq',
 'saskylakh',
 'necochea',
 'east london',
 'taolanaro',
 'ivolginsk',
 'vaini',
 'thompson',
 'cabo san lucas',
 'mount gambier',
 'puerto ayora',
 'georgetow

In [7]:
requests.get(query_url+'chuy').json()["coord"]["lat"]

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
#use API to scan through the dataset and insert it into the df and print a summary
number = 0
group = 1

#limit the API search so it doesn't max out


#start API run
print ("Initiate API Data Retrieval")
print ("-------------------------------")

for place in cities:
    try:
        print (f"Processing the file {number} of Group {group} | {place}.")
            
        #place the response into variables first
        response = requests.get(query_url+place).json()
        
        city_list.append(place)
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        temp.append(response["main"]["temp_max"])
        hum.append(response["main"]["humidity"])
        cld.append(response["clouds"]["all"])
        wind.append(response["wind"]["speed"])
        dt.append(response["dt"])
        country.append(response["sys"]["country"])

        
    except:
        (f"Weather for the city is unavaliable, checking next city: {place}.")
        
        no_data.remove(cities)    
            
    number = number + 1
    if number == 50:
        number= 0
        group = group + 1   
        
print ("-------------------------------")
print ("API Retrieval is complete")
print(API_url)
print(f"city name: {city_list}")
print (f"The latitude information received is: {lat}")
print(f"The temperature information received is: {temp}")            

        
        

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
print (len(lng))

In [ ]:



weather_by_city_dict = {"City": place, "Country": country, "Latitude": lat, "Longtitude": lng, "Max Temp": temp, "Humidity": hum, "Cloudiness": cld, "Wind Speed": wind, "Date":dt}

weather_by_city_df = pd.DataFrame(weather_by_city_dict)


weather_by_city_df

In [ ]:
weather_by_city_df.to_csv("../output_data/weather_data.csv")

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(pd.to_numeric(weather_df["Latitude"]).astype(float), pd.to_numeric(weather_df["Max Temp (F)"]).astype(float),  alpha=.5, facecolor="red", edgecolors="black", marker ="o")
plt.title(f"Latitude vs Max Temperature in F")
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)") 
print(f"The max temperatures are highest near the equator.")
plt.show()

In [ ]:
# Save figure to Images folder dont run this right now ##########
plt.savefig("Images/Latitude vs Temperature.png")

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(pd.to_numeric(city_weather["Latitude"]).astype(float), pd.to_numeric(city_weather["Humidity (%)"]).astype(float), facecolors="skyblue", edgecolors="purple", alpha = 0.5)
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.title(f"Latitude vs Humidity as at 14/09/2021")
plt.show()

In [ ]:
plt.savefig("Images/Latitude vs Humidity.png")

## Latitude vs. Cloudiness Plot

In [ ]:
plt.savefig("Images/latitude_cloudiness.png")

## Latitude vs. Wind Speed Plot

In [ ]:
plt.savefig("Images/latitude vs wind speed.png")

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression